# Q2: Data Cleaning

**Phase 3:** Data Cleaning & Preprocessing  
**Points: 9 points**

**Focus:** Handle missing data, outliers, validate data types, remove duplicates.

**Lecture Reference:** Lecture 11, Notebook 1 ([`11/demo/01_setup_exploration_cleaning.ipynb`](https://github.com/christopherseaman/datasci_217/blob/main/11/demo/01_setup_exploration_cleaning.ipynb)), Phase 3. Also see Lecture 05 (data cleaning).

---

## Setup

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

os.makedirs("output", exist_ok=True)

df = pd.read_csv("data/beach_sensors.csv")

rows_before = len(df)

datetime_cols = []
for col in df.columns:
    if ('time' in col.lower()) or ('date' in col.lower()):
        try:
            df[col] = pd.to_datetime(df[col])
            datetime_cols.append(col)
        except Exception:
            pass

if datetime_cols:
    df = df.sort_values(by=datetime_cols[0])

missing_before = df.isna().sum()
missing_info = []

numeric_cols = df.select_dtypes(include=[np.number]).columns
non_numeric_cols = [c for c in df.columns if c not in numeric_cols]

for col in numeric_cols:
    col_missing_before = df[col].isna().sum()
    if col_missing_before == 0:
        continue

    df[col] = df[col].fillna(method="ffill")

    remaining_missing = df[col].isna().sum()
    if remaining_missing > 0:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

    col_missing_after = df[col].isna().sum()
    values_filled = col_missing_before - col_missing_after

    missing_info.append({
        "column": col,
        "missing_before": int(col_missing_before),
        "missing_after": int(col_missing_after),
        "values_filled": int(values_filled),
        "method": "forward-fill, then median imputation"
    })

for col in non_numeric_cols:
    col_missing_before = df[col].isna().sum()
    if col_missing_before == 0:
        continue

    df[col] = df[col].fillna(method="ffill")
    df[col] = df[col].fillna(method="bfill")

    col_missing_after = df[col].isna().sum()
    values_filled = col_missing_before - col_missing_after

    missing_info.append({
        "column": col,
        "missing_before": int(col_missing_before),
        "missing_after": int(col_missing_after),
        "values_filled": int(values_filled),
        "method": "forward-fill, then back-fill"
    })

outlier_info = []

for col in numeric_cols:
    series = df[col].copy()
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1

    if pd.isna(IQR) or IQR == 0:
        continue

    lower = Q1 - 3 * IQR
    upper = Q3 + 3 * IQR

    mask_outliers = (series < lower) | (series > upper)
    num_outliers = mask_outliers.sum()

    if num_outliers > 0:
        df[col] = np.clip(series, lower, upper)

        outlier_info.append({
            "column": col,
            "num_outliers": int(num_outliers),
            "method": "capped using 3*IQR",
            "lower_bound": float(lower),
            "upper_bound": float(upper)
        })

duplicates_before = df.duplicated().sum()
df = df.drop_duplicates()
duplicates_removed = duplicates_before
rows_after = len(df)

report_lines = []
report_lines.append("DATA CLEANING REPORT")
report_lines.append("====================\n")
report_lines.append(f"Rows before cleaning: {rows_before}\n")


report_lines.append("Missing Data Handling:")
if not missing_info:
    report_lines.append("- No missing values detected.\n")
else:
    for info in missing_info:
        report_lines.append(
            f"- {info['column']}: {info['missing_before']} missing values before, "
            f"{info['missing_after']} after cleaning"
        )
        report_lines.append(f"  Method: {info['method']}")
        report_lines.append(f"  Result: {info['values_filled']} values filled\n")

report_lines.append("Outlier Handling:")
if not outlier_info:
    report_lines.append("- No numeric outliers handled (IQR method found none or columns constant).\n")
else:
    for info in outlier_info:
        report_lines.append(
            f"- {info['column']}: Detected {info['num_outliers']} outliers using 3*IQR"
        )
        report_lines.append(
            f"  Method: {info['method']}"
        )
        report_lines.append(
            f"  Bounds: [{info['lower_bound']:.3f}, {info['upper_bound']:.3f}]"
        )
        report_lines.append(
            f"  Result: {info['num_outliers']} values capped\n"
        )

report_lines.append(f"Duplicates Removed: {duplicates_removed}\n")

report_lines.append("Data Type Conversions:")
if datetime_cols:
    for col in datetime_cols:
        report_lines.append(f"- {col}: Converted to datetime64[ns]")
else:
    report_lines.append("- No datetime conversions applied.")

report_lines.append("")
report_lines.append(f"Rows after cleaning: {rows_after}")

report_text = "\n".join(report_lines)


df.to_csv("output/q2_cleaned_data.csv", index=False)

with open("output/q2_cleaning_report.txt", "w") as f:
    f.write(report_text)

with open("output/q2_rows_cleaned.txt", "w") as f:
    f.write(str(rows_after))

print("Cleaning completed.")
print(f"Rows before: {rows_before}, rows after: {rows_after}")

/tmp/ipykernel_30779/1481575100.py:37: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[col] = df[col].fillna(method="ffill")


Cleaning completed.
Rows before: 196453, rows after: 196453


---

## Objective

Clean the dataset by handling missing data, outliers, validating data types, and removing duplicates.

**Time Series Note:** For time series data, forward-fill (`ffill()`) is often appropriate for missing values since sensor readings are continuous. However, you may choose other strategies based on your analysis.

---

## Required Artifacts

You must create exactly these 3 files in the `output/` directory:

### 1. `output/q2_cleaned_data.csv`
**Format:** CSV file
**Content:** Cleaned dataset with same structure as original (same columns)
**Requirements:**
- Same columns as original dataset
- Missing values handled (filled, dropped, or imputed)
- Outliers handled (removed, capped, or transformed)
- Data types validated and converted
- Duplicates removed
- **Sanity check:** Dataset should retain most rows after cleaning (at least 1,000 rows). If you're removing more than 50% of data, reconsider your strategy—imputation is usually preferable to dropping rows for this dataset.
- **No index column** (save with `index=False`)

### 2. `output/q2_cleaning_report.txt`
**Format:** Plain text file
**Content:** Detailed report of cleaning operations
**Required information:**
- Rows before cleaning: [number]
- Missing data handling method: [description]
  - Which columns had missing data
  - Method used (drop, forward-fill, impute, etc.)
  - Number of values handled
- Outlier handling: [description]
  - Detection method (IQR, z-scores, domain knowledge)
  - Which columns had outliers
  - Method used (remove, cap, transform)
  - Number of outliers handled
- Duplicates removed: [number]
- Data type conversions: [list any conversions]
- Rows after cleaning: [number]

**Example format:**
```
DATA CLEANING REPORT
====================

Rows before cleaning: 50000

Missing Data Handling:
- Water Temperature: 2500 missing values (5.0%)
  Method: Forward-fill (time series appropriate)
  Result: All missing values filled
  
- Air Temperature: 1500 missing values (3.0%)
  Method: Forward-fill, then median imputation for remaining
  Result: All missing values filled

Outlier Handling:
- Water Temperature: Detected 500 outliers using IQR method (3×IQR)
  Method: Capped at bounds [Q1 - 3×IQR, Q3 + 3×IQR]
  Bounds: [-5.2, 35.8]
  Result: 500 values capped

Duplicates Removed: 0

Data Type Conversions:
- Measurement Timestamp: Converted to datetime64[ns]

Rows after cleaning: 50000
```

### 3. `output/q2_rows_cleaned.txt`
**Format:** Plain text file
**Content:** Single integer number (total rows after cleaning)
**Requirements:**
- Only the number, no text, no labels
- No whitespace before or after
- Example: `50000`

---

## Requirements Checklist

- [ ] Missing data handling strategy chosen and implemented
- [ ] Outliers detected and handled (IQR method, z-scores, or domain knowledge)
- [ ] Data types validated and converted
- [ ] Duplicates identified and removed
- [ ] Cleaning decisions documented in report
- [ ] All 3 required artifacts saved with exact filenames

---

## Your Approach

1. **Handle missing data** - Choose appropriate strategy (drop, forward-fill, impute) based on data characteristics
2. **Detect and handle outliers** - Use IQR method or z-scores; decide whether to remove, cap, or transform
3. **Validate data types** - Ensure numeric and datetime columns are properly typed
4. **Remove duplicates**
5. **Document and save** - Write detailed cleaning report explaining your decisions

---

## Decision Points

- **Missing data:** Should you drop rows, impute values, or forward-fill? Consider: How much data is missing? Is it random or systematic? For time series, forward-fill is often appropriate.
- **Outliers:** Are they errors or valid extreme values? Use IQR method or z-scores to detect, then decide: remove, cap, or transform. Document your reasoning.
- **Data types:** Are numeric columns actually numeric? Are datetime columns properly formatted? Convert as needed.

---

## Checkpoint

After Q2, you should have:
- [ ] Missing data handled
- [ ] Outliers addressed
- [ ] Data types validated
- [ ] Duplicates removed
- [ ] All 3 artifacts saved: `q2_cleaned_data.csv`, `q2_cleaning_report.txt`, `q2_rows_cleaned.txt`

---

**Next:** Continue to `q3_data_wrangling.md` for Data Wrangling.
